Load data

In [1]:
import xarray as xr
from datetime import datetime
from methods.method1.spei_calc_multi import generate_t2m_filenames, generate_imerg_filenames

# Path to the netCDF files
temp_file_paths = generate_t2m_filenames(
    start_date=datetime(2013, 8, 1),
    end_date=datetime(2023, 8, 1),
    dir="/media/jtrvz/1tb/drought_data/temperature/era5/Global/monthly/netcdf/avg",
)
prec_file_paths = generate_imerg_filenames(
    start_date=datetime(2013, 8, 1),
    end_date=datetime(2023, 8, 1),
    dir="/media/jtrvz/1tb/drought_data/precipitation/nasa_gpm/Global/monthly/netcdf/avg",
)

print(len(temp_file_paths))
print(len(prec_file_paths))

121
121


In [2]:
from methods.method1.spei_calc_multi import read_nc_files
# Load the precipitation and temperature data
prec_ds = read_nc_files(prec_file_paths)
temp_ds = read_nc_files(temp_file_paths)

In [3]:
prec_ds

<xarray.Dataset> Size: 16GB
Dimensions:                         (time: 121, lon: 3600, lat: 1800, latv: 2,
                                     nv: 2, lonv: 2)
Coordinates:
  * lat                             (lat) float32 7kB -89.95 -89.85 ... 89.95
  * latv                            (latv) int32 8B 0 1
  * lon                             (lon) float32 14kB -179.9 -179.9 ... 179.9
  * lonv                            (lonv) int32 8B 0 1
  * nv                              (nv) int32 8B 0 1
  * time                            (time) object 968B 2013-08-01 00:00:00 .....
Data variables:
    precipitationQualityIndex       (time, lon, lat) float32 3GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    gaugeRelativeWeighting          (time, lon, lat) float32 3GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    lat_bnds                        (time, lat, latv) float32 2MB dask.array<chunksize=(1, 1024, 2), meta=np.ndarray>
    precipitation                   (time, lon, lat) float32 3GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    probabilityLiquidPrecipitation  (time, lon, lat) float32 3GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    randomError                     (time, lon, lat) float32 3GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    time_bnds                       (time, nv) object 2kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lon_bnds                        (time, lon, lonv) float32 3MB dask.array<chunksize=(1, 1024, 2), meta=np.ndarray>
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-MONTH/07;\nDOIa...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:                         2024-02-20 12:10:19 GMT Hyrax-1.16.3 htt...

In [4]:
temp_ds

,Array,Chunk
Bytes,11.69 GiB,98.93 MiB
Shape,"(121, 2, 1801, 3600)","(1, 2, 1801, 3600)"
Count,363 Tasks,121 Chunks
Type,float64,numpy.ndarray


Data Preprocessing

In [5]:
from methods.method1.spei_calc_multi import preprocess_temp, preprocess_prec, spatial_subset
# Spatial subset for Germany (Europe)
lat_bounds = [47.0, 55.0]
lon_bounds = [5.5, 15.0]

In [6]:
# Extract the necessary variables, adjust names as per your dataset
# Rearrange the dimensions of the precipitation xarray.DataArray
prec_ds = preprocess_prec(prec_ds)
prec_ds = spatial_subset(prec_ds, lat_bounds, lon_bounds)
prec_ds

<xarray.Dataset> Size: 19MB
Dimensions:                         (time: 121, lat: 80, lon: 95, latv: 2,
                                     nv: 2, lonv: 2)
Coordinates:
  * lat                             (lat) float32 320B 47.05 47.15 ... 54.95
  * latv                            (latv) int32 8B 0 1
  * lon                             (lon) float32 380B 5.55 5.65 ... 14.85 14.95
  * lonv                            (lonv) int32 8B 0 1
  * nv                              (nv) int32 8B 0 1
  * time                            (time) datetime64[ns] 968B 2013-08-01 ......
Data variables:
    precipitationQualityIndex       (time, lat, lon) float32 4MB dask.array<chunksize=(1, 80, 95), meta=np.ndarray>
    gaugeRelativeWeighting          (time, lat, lon) float32 4MB dask.array<chunksize=(1, 80, 95), meta=np.ndarray>
    lat_bnds                        (time, lat, latv) float32 77kB dask.array<chunksize=(1, 80, 2), meta=np.ndarray>
    precipitation                   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 80, 95), meta=np.ndarray>
    probabilityLiquidPrecipitation  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 80, 95), meta=np.ndarray>
    randomError                     (time, lat, lon) float32 4MB dask.array<chunksize=(1, 80, 95), meta=np.ndarray>
    time_bnds                       (time, nv) object 2kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lon_bnds                        (time, lon, lonv) float32 92kB dask.array<chunksize=(1, 95, 2), meta=np.ndarray>
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-MONTH/07;\nDOIa...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:                         2024-02-20 12:10:19 GMT Hyrax-1.16.3 htt...

In [7]:
temp_ds = preprocess_temp(temp_ds)
temp_ds = spatial_subset(temp_ds, lat_bounds, lon_bounds)
temp_ds

/media/jtrvz/chugchug/Git/drought-detector/venv/lib/python3.10/site-packages/xarray/core/indexing.py:1593: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


,Array,Chunk
Bytes,14.36 MiB,121.50 kiB
Shape,"(121, 2, 81, 96)","(1, 2, 81, 96)"
Count,847 Tasks,121 Chunks
Type,float64,numpy.ndarray


In [8]:
from methods.method1.spei_calc_multi import calc_pet_thornthwaite
# Apply the Thornthwaite method to estimate PET
pet_ds = pet_germany = calc_pet_thornthwaite(temp_ds)
pet_ds

,Array,Chunk
Bytes,14.36 MiB,121.50 kiB
Shape,"(121, 2, 81, 96)","(1, 2, 81, 96)"
Count,2905 Tasks,121 Chunks
Type,float64,numpy.ndarray


In [9]:
# Preprocess the PET dataset
from methods.method1.spei_calc_multi import preprocess_pet
pet_ds = preprocess_pet(pet_ds, prec_ds)

In [10]:
pet_ds

<xarray.DataArray (time: 121, lat: 80, lon: 95)> Size: 7MB
dask.array<chunked_aware_interpnd, shape=(121, 80, 95), dtype=float64, chunksize=(1, 80, 95), chunktype=numpy.ndarray>
Coordinates:
    expver   int32 4B 1
  * time     (time) datetime64[ns] 968B 2013-08-01 2013-09-01 ... 2023-08-01
  * lat      (lat) float32 320B 47.05 47.15 47.25 47.35 ... 54.75 54.85 54.95
  * lon      (lon) float32 380B 5.55 5.65 5.75 5.85 ... 14.65 14.75 14.85 14.95

In [11]:
print(pet_ds.dims)
print(prec_ds.dims)

('time', 'lat', 'lon')
FrozenMappingWarningOnValuesAccess({'time': 121, 'lat': 80, 'lon': 95, 'latv': 2, 'nv': 2, 'lonv': 2})


Compute water balance

In [12]:
from methods.method1.spei_calc_multi import calc_difference
# Calculate water balance as difference between precipitation and PET
diff_ds = calc_difference(prec_ds, pet_ds)

In [13]:
diff_ds

<xarray.DataArray (time: 121, lat: 80, lon: 95)> Size: 7MB
dask.array<sub, shape=(121, 80, 95), dtype=float64, chunksize=(1, 80, 95), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 320B 47.05 47.15 47.25 47.35 ... 54.75 54.85 54.95
  * lon      (lon) float32 380B 5.55 5.65 5.75 5.85 ... 14.65 14.75 14.85 14.95
  * time     (time) datetime64[ns] 968B 2013-08-01 2013-09-01 ... 2023-08-01
    expver   int32 4B 1

In [14]:
# import numpy as np
# import xarray as xr

# def validate_data_count(D, original_shape):
#     valid_data_count = np.sum(np.isfinite(D))
#     expected_count = np.prod(original_shape)
#     print(f"Valid data count: {valid_data_count}, Expected count: {expected_count}")

# # Example usage
# validate_data_count(diff_ds.values.flatten(), (121, 80, 95))

Calculate SPEI

In [15]:
from methods.method1.spei_calc_multi import calc_spei
spei_ds = calc_spei(diff_ds, "ll")

/media/jtrvz/chugchug/Git/drought-detector/venv/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in power
  return func(*(_execute_task(a, cache) for a in args))


In [16]:
type(spei_ds)

xarray.core.dataarray.DataArray

Prepare Data for ML

In [18]:
import pandas as pd

# Convert DataArrays to DataFrame
pet_df = pet_ds.to_dataframe(name='PET')
diff_df = diff_ds.to_dataframe(name='Water_Balance')
prec_df = prec_ds.to_dataframe()
temp_df = temp_ds.to_dataframe()
spei_df = spei_ds.to_dataframe(name="SPEI")

/media/jtrvz/chugchug/Git/drought-detector/venv/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in power
  return func(*(_execute_task(a, cache) for a in args))


In [19]:
# Reset index to turn the multi-index into columns
pet_df.reset_index(inplace=True)
diff_df.reset_index(inplace=True)
prec_df.reset_index(inplace=True)
temp_df.reset_index(inplace=True)
spei_df.reset_index(inplace=True)

In [21]:
# Merge DataFrames
df = pd.merge(pet_df, diff_df, on=["time", "lat", "lon"], how="inner")
df = pd.merge(df, prec_df, on=["time", "lat", "lon"], how="inner", suffixes=('', '_prec'))
df = pd.merge(df, temp_df, on=["time", "lat", "lon"], how="inner", suffixes=('', '_temp'))
df = pd.merge(df, spei_df, on=["time", "lat", "lon"], how="inner", suffixes=('', '_spei'))


In [22]:
# Adding time features such as month and year
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year

In [23]:
# Simple feature engineering: add time components as features
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year

In [24]:
df

,time,lat,lon,expver_x,PET,expver_y,Water_Balance,latv,nv,lonv,...,probabilityLiquidPrecipitation,randomError,time_bnds,lon_bnds,expver,t2m,expver_spei,SPEI,month,year


Train and Test Split

In [26]:
from sklearn.model_selection import train_test_split

# Define features and target variable
features = df[['PET', 'Water_Balance', 'precipitation', 't2m', 'month', 'year']]
target = df['SPEI']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

Apply GBT

In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target variable
features = df[['PET', 'Water_Balance', 'Precipitation', 'Temperature', 'month', 'year']]
target = df['SPEI']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
